# 🧠 Segment 2 — Activation Maximization (InceptionV1)

This notebook generates **activation maximization images** for every neuron
in a chosen InceptionV1 layer using [Lucent](https://github.com/greentfrapp/lucent).

## What this notebook does
1. Loads a pretrained InceptionV1 model
2. For each neuron in a layer, optimizes a random image to maximally activate it
3. Saves the resulting visualization as a `.png` file (lossless)
4. Logs all metrics and images to [Weights & Biases](https://wandb.ai)

## Robustness features
- **Resume capability** — re-running skips already-completed neurons
- **GPU memory management** — periodic cleanup prevents OOM on long runs
- **Error isolation** — a single neuron failure won’t crash the entire run
- **Disk error handling** — graceful skip if disk write fails

In [1]:
# =============================================================================
# Cell 2: Environment Setup
# =============================================================================
# Detects whether we are running on Google Colab or locally.
# Sets the CUDA memory allocator to use expandable segments, which prevents
# GPU memory fragmentation during long runs (800+ neuron iterations).
#
# ROBUSTNESS NOTE (#2 — CUDA Fragmentation):
#   expandable_segments:True must be set BEFORE importing torch.
#   Without this, after ~400-500 neurons the allocator may report
#   "CUDA out of memory" even though nvidia-smi shows free memory.
# =============================================================================

import sys
import os

# --- CUDA allocator config (must be before torch import) ---
# This tells PyTorch to use expandable memory segments instead of
# fixed-size blocks. It prevents the "reserved but unallocated" OOM
# pattern that appears in long-running loops.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- Environment detection ---
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Colab-specific: install lucent if not available
    pass
else:
    # Local setup: add project source directory to Python path
    from pathlib import Path

    project_root = Path.cwd().parent  # notebooks/ → project root
    sys.path.insert(0, str(project_root / "src"))
    print(f"✅ Local setup complete (project root: {project_root})")

✅ Local setup complete (project root: c:\Users\cataluna84\Documents\Workspace\VisionInterpretability)


In [2]:
# =============================================================================
# Cell 3: Imports & Configuration
# =============================================================================
# All imports are grouped by category (PEP-8 style):
#   1. Standard library
#   2. Third-party libraries
#   3. Project-specific / Lucent
#
# The CONFIG dictionary centralizes every tunable parameter so you
# only need to edit one place to change the run.
# =============================================================================

# --- Standard library ---
import gc                    # Garbage collector — forces Python to free memory
import time                  # Wall-clock timing for throughput measurement

# --- Third-party ---
import torch                 # PyTorch — GPU tensor computation
import numpy as np           # NumPy — array operations for image processing
import matplotlib.pyplot as plt  # Matplotlib — (optional) for inline plots
from pathlib import Path     # Pathlib — cross-platform file path handling
from PIL import Image        # Pillow — PNG/JPEG image saving
from tqdm.auto import tqdm   # tqdm — progress bar with ETA and throughput
from dotenv import load_dotenv  # dotenv — loads .env file for API keys

# --- Lucent (activation maximization library) ---
from lucent.optvis import render, param   # render_vis + image parameterization
from lucent.modelzoo import inceptionv1   # Pretrained InceptionV1 model

# --- Load environment variables (local only) ---
# The .env file should contain your WANDB_API_KEY
if not IN_COLAB:
    load_dotenv(project_root / ".env")

# --- Device selection ---
# Uses GPU if available, otherwise falls back to CPU.
# For 832+ neurons at 512px, a GPU is strongly recommended (~30s/neuron on GPU
# vs ~10min/neuron on CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Using device: {device}")
if device.type == "cuda":
    print(f"   GPU name: {torch.cuda.get_device_name()}")
    print(f"   GPU memory: {torch.cuda.get_device_properties(0).total_memory / 2**30:.1f} GB")

# --- Configuration dictionary ---
# Edit these values to customize your run.
# All other cells read from this dictionary.
CONFIG = {
    # Which layer to visualize (e.g., "mixed4a", "mixed5a", "mixed5b")
    "layer": "mixed5a",

    # Which neurons to process — range(0, 832) for mixed5a, range(0, 1024) for mixed5b
    "neurons": list(range(0, 832)),

    # Output image resolution in pixels (higher = more detail, more memory)
    "image_size": 512,

    # Number of optimization steps per neuron (higher = sharper features)
    "num_steps": 1024,

    # How often to flush GPU memory (every N neurons)
    # Lower = more frequent cleanup = slightly slower but safer
    "memory_cleanup_interval": 50,

    # Weights & Biases project/run naming
    "wandb_project": "vision-interpretability",
    "wandb_run_name": "mixed5a-actmax-512px",

    # Image format: "png" for lossless, "jpg" for smaller files
    "image_format": "png",

    # PNG compress level (0-9, lower = faster but larger files)
    # Only used when image_format is "png"
    "png_compress_level": 1,
}

🖥️  Using device: cuda
   GPU name: NVIDIA GeForce RTX 2070
   GPU memory: 8.0 GB


In [ ]:
# =============================================================================
# Cell 4: Auto-Increment Run ID
# =============================================================================
# Creates a sequential run ID (001, 002, 003, ...) by scanning the results
# directory for existing run folders.
#
# ROBUSTNESS NOTE (#9 — Multi-Layer Runs):
#   Each run gets its own numbered directory, and within it, each layer
#   gets a subdirectory. This means you can run mixed5a then mixed5b
#   under the same run ID, or give each its own run ID.
#
# Directory structure:
#   results/segment_2_activation_max/
#   ├── 001/
#   │   └── mixed5a/
#   │       ├── 0.png
#   │       ├── 1.png
#   │       └── ...
#   └── 002/
#       └── mixed5b/
#           └── ...
# =============================================================================

# --- Choose base directory based on environment ---
if IN_COLAB:
    _results_base = Path("/content/drive/MyDrive/activation_max_results")
else:
    _results_base = project_root / "notebooks" / "results" / "segment_2_activation_max"

# --- Create base directory if it doesn't exist ---
_results_base.mkdir(parents=True, exist_ok=True)

# --- Find the next sequential run ID ---
# Scan for folders named "001", "002", etc. and pick the next number.
_existing_ids = [
    int(p.name)
    for p in _results_base.iterdir()
    if p.is_dir() and p.name.isdigit()
]
run_id = "002" # f"{max(_existing_ids) + 1:03d}" if _existing_ids else "001"

# --- Create the output directory for this run + layer ---
save_dir = _results_base / run_id / CONFIG["layer"]
save_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Run ID:    {run_id}")
print(f"📁 Save dir:  {save_dir}")

📁 Run ID:    002
📁 Save dir:  c:\Users\cataluna84\Documents\Workspace\VisionInterpretability\notebooks\results\segment_2_activation_max\002\mixed5a


In [4]:
# =============================================================================
# Cell 5: Initialize Weights & Biases
# =============================================================================
# W&B tracks metrics, images, and system stats for every run.
# You can view your runs at: https://wandb.ai/<your-username>/vision-interpretability
#
# ROBUSTNESS NOTE (#6 — W&B Network Resilience):
#   init_timeout=120 gives W&B 2 minutes to connect (default is 60s).
#   If the network drops mid-run, W&B buffers data locally and syncs
#   when reconnected — no data is lost.
# =============================================================================

import wandb

wandb.init(
    project=CONFIG["wandb_project"],
    name=f"{CONFIG['wandb_run_name']}-run{run_id}",
    config=CONFIG,
    settings=wandb.Settings(
        init_timeout=120,  # Allow 2 min for init on slow networks
    ),
)

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: cataluna84 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
# =============================================================================
# Cell 6: Load Pretrained InceptionV1
# =============================================================================
# Loads InceptionV1 with ImageNet weights and freezes all parameters.
# We freeze parameters because we are NOT training the model — we only
# need the forward pass to measure neuron activations.
# =============================================================================

# --- Load and configure model ---
model = inceptionv1(pretrained=True)  # Download ImageNet weights
model = model.to(device)              # Move to GPU (if available)
model = model.eval()                  # Set to evaluation mode (disables dropout, etc.)

# --- Freeze all parameters ---
# This saves memory because PyTorch won't store gradients for frozen params.
for p in model.parameters():
    p.requires_grad_(False)

print(f"✅ InceptionV1 loaded on {device} (all parameters frozen)")

✅ InceptionV1 loaded on cuda (all parameters frozen)


c:\Users\cataluna84\Documents\Workspace\VisionInterpretability\.venv\Lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


## Activation Maximization Function

The function below optimizes a random noise image so that it **maximally activates**
a specific neuron in the network. This is done via gradient ascent on the input image
(not the model weights).

**How it works:**
1. Start with a random image parameterized in Fourier space (`fft=True`)
2. Run it through the model and measure the target neuron’s activation
3. Compute gradients of the activation with respect to the image pixels
4. Update the image to increase the activation
5. Repeat for `num_steps` iterations

The `decorrelate=True` flag applies a color decorrelation transform that
produces more natural-looking colors in the output.

In [6]:
# =============================================================================
# Cell 8: Activation Maximization Function
# =============================================================================
# This function wraps Lucent's render_vis() to generate one activation
# maximization image for a single neuron.
#
# Parameters:
#   model       — the pretrained InceptionV1 model
#   layer_name  — which layer to target (e.g., "mixed5a")
#   neuron_id   — which neuron/channel in that layer (e.g., 42)
#   image_size  — output resolution in pixels (default 512)
#   num_steps   — number of optimization iterations (default 1024)
#
# Returns:
#   numpy array of shape (H, W, 3) with values in [0, 1]
# =============================================================================


def activation_maximization(
    model,
    layer_name,
    neuron_id,
    image_size=512,
    num_steps=1024,
):
    """Generate an activation maximization image for a single neuron.

    Uses Lucent's render_vis with FFT parameterization and color
    decorrelation for visually interpretable results.

    Args:
        model: Pretrained PyTorch model (e.g., InceptionV1).
        layer_name: Target layer name (e.g., "mixed5a").
        neuron_id: Target neuron/channel index within the layer.
        image_size: Output image resolution in pixels. Default 512.
        num_steps: Number of gradient ascent steps. Default 1024.

    Returns:
        np.ndarray: Image array of shape (H, W, 3), values in [0, 1].
    """
    # Build the objective string that Lucent understands
    # Format: "layer_name:neuron_index"
    objective = f"{layer_name}:{neuron_id}"

    # Define the image parameterization:
    #   fft=True        → optimize in Fourier space (smoother results)
    #   decorrelate=True → apply color decorrelation (more natural colors)
    param_f = lambda: param.image(
        image_size,
        fft=True,
        decorrelate=True,
    )

    # Run the optimization loop
    #   thresholds=(num_steps,) → only return the final image, not intermediates
    #   show_image=False        → don't display inline (we save to disk instead)
    #   verbose=False           → suppress Lucent's own progress output
    images = render.render_vis(
        model,
        objective,
        param_f=param_f,
        thresholds=(num_steps,),
        show_image=False,
        verbose=False,
    )

    # images is a list of [batch of images] per threshold
    # images[0] = results at our single threshold
    # images[0][0] = first (and only) image in the batch
    return images[0][0]

## Generate & Save All Neuron Visualizations

The cell below is the **main execution loop**. It processes every neuron in
`CONFIG["neurons"]` and saves the resulting image to disk.

### Robustness features built into this loop:
| Feature | What it does |
|---------|--------------|
| **Resume** | Skips neurons that already have a `.png` file (lossless) saved |
| **Memory cleanup** | Runs `gc.collect()` + `torch.cuda.empty_cache()` every 50 neurons |
| **OOM recovery** | If a neuron fails, clears GPU memory before trying the next one |
| **NaN/Inf guard** | Detects degenerate outputs and skips them |
| **Disk error handling** | Catches write failures without crashing |
| **Progress bar** | Shows ETA, speed, and failure count in real time |
| **W&B logging** | Logs timing, images, and GPU memory to your dashboard |

In [ ]:
# =============================================================================
# Cell 10: Generate & Save All Neuron Visualizations
# =============================================================================
# Main execution loop with full robustness for 7-8 hour runs.
# See the markdown cell above for a summary of all safety features.
# =============================================================================

# -------------------------------------------------------------------------
# Timing accumulators — track where time is spent across the run
# -------------------------------------------------------------------------
total_time = 0.0          # Will be computed at the end
time_optimization = 0.0   # Cumulative time spent in render_vis()
time_saving = 0.0         # Cumulative time spent writing image files
neurons_completed = 0     # Count of successfully saved neurons
neurons_failed = 0        # Count of neurons that errored or produced NaN

# -------------------------------------------------------------------------
# ROBUSTNESS #8 — Resume after crash
# -------------------------------------------------------------------------
# Scan the save directory for image files that already exist.
# If the notebook was interrupted at hour 5, restarting it will
# seamlessly skip to the first uncompleted neuron.
_ext = CONFIG["image_format"]  # "png" or "jpg"
_existing = {int(f.stem) for f in save_dir.glob(f"*.{_ext}")}
neurons_to_process = [n for n in CONFIG["neurons"] if n not in _existing]
_skipped = len(CONFIG["neurons"]) - len(neurons_to_process)

if _skipped:
    print(f"⏭️  Skipping {_skipped} already-completed neurons (found in {save_dir})")

# -------------------------------------------------------------------------
# Progress bar setup
# -------------------------------------------------------------------------
# tqdm provides a live progress bar with:
#   - elapsed time and ETA
#   - processing speed (neurons/sec)
#   - custom postfix showing last/avg time and failure count
pbar = tqdm(
    neurons_to_process,
    desc=f"🧠 {CONFIG['layer']}",
    unit="neuron",
    dynamic_ncols=True,
    bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]",
)

# Record the start time for total elapsed calculation
t_start_total = time.time()

# -------------------------------------------------------------------------
# Main loop — one iteration per neuron
# -------------------------------------------------------------------------
for neuron_id in pbar:
    t_neuron_start = time.time()

    try:
        # -----------------------------------------------------------------
        # Step 1: Run activation maximization
        # -----------------------------------------------------------------
        # ROBUSTNESS #10 — torch.no_grad() wrapper
        # Lucent manages its own gradients internally. This wrapper
        # prevents any accidental computational graph construction
        # in our outer code, saving memory.
        t_opt_start = time.time()

        with torch.no_grad():
            img = activation_maximization(
                model,
                CONFIG["layer"],
                neuron_id,
                image_size=CONFIG["image_size"],
                num_steps=CONFIG["num_steps"],
            )

        opt_time = time.time() - t_opt_start
        time_optimization += opt_time

        # -----------------------------------------------------------------
        # Step 2: Safety — detach tensor if Lucent returned one
        # -----------------------------------------------------------------
        # ROBUSTNESS #3 — Computational graph leak prevention
        # Normally Lucent returns a numpy array, but as a safety net
        # we check and detach if it's a tensor. This prevents the
        # computational graph from being pinned in GPU memory.
        if isinstance(img, torch.Tensor):
            img = img.detach().cpu().numpy()

        # -----------------------------------------------------------------
        # Step 3: Check for NaN or Inf values
        # -----------------------------------------------------------------
        # ROBUSTNESS #4 — NaN/Inf detection
        # Some neurons may produce degenerate optimizations. Rather than
        # saving a corrupt image, we log it and move on.
        if np.any(np.isnan(img)) or np.any(np.isinf(img)):
            print(f"\n⚠️ Neuron {neuron_id}: NaN/Inf detected, skipping save")
            wandb.log({"neuron": neuron_id, "error": "NaN/Inf in output"})
            neurons_failed += 1
            del img  # Free the bad array
            continue

        # -----------------------------------------------------------------
        # Step 4: Convert and save to disk
        # -----------------------------------------------------------------
        # ROBUSTNESS #5 — Disk I/O error handling
        # Convert float [0,1] → uint8 [0,255] and save as PNG (lossless).
        # Wrapped in try/except to handle disk-full or permission errors.
        t_save_start = time.time()
        img_uint8 = (np.clip(img, 0, 1) * 255).astype(np.uint8)
        save_path = save_dir / f"{neuron_id}.{CONFIG['image_format']}"

        try:
            # PNG: use compress_level for speed/size tradeoff (lossless)
            # JPG: use quality=100 for minimum compression
            if CONFIG["image_format"] == "png":
                Image.fromarray(img_uint8).save(
                    save_path, compress_level=CONFIG["png_compress_level"]
                )
            else:
                Image.fromarray(img_uint8).save(save_path, quality=100)
        except OSError as e:
            print(f"\n⚠️ Neuron {neuron_id}: Disk write failed: {e}")
            wandb.log({"neuron": neuron_id, "error": f"disk: {e}"})
            neurons_failed += 1
            del img, img_uint8  # Clean up before continuing
            continue

        save_time = time.time() - t_save_start
        time_saving += save_time

        # -----------------------------------------------------------------
        # Step 5: Record success
        # -----------------------------------------------------------------
        neuron_time = time.time() - t_neuron_start
        neurons_completed += 1

        # -----------------------------------------------------------------
        # Step 6: Log metrics and image to W&B
        # -----------------------------------------------------------------
        wandb.log({
            "neuron": neuron_id,
            "optimization_time_sec": opt_time,
            "save_time_sec": save_time,
            "total_neuron_time_sec": neuron_time,
            "neurons_completed": neurons_completed,
            "neurons_failed": neurons_failed,
            "elapsed_total_sec": time.time() - t_start_total,
            "image": wandb.Image(
                img_uint8,
                caption=f"{CONFIG['layer']}/n{neuron_id}",
            ),
        })

        # -----------------------------------------------------------------
        # Step 7: Free large arrays
        # -----------------------------------------------------------------
        # ROBUSTNESS #1 — Explicit memory lifecycle
        # Delete references to the image arrays so Python can free them.
        # This is especially important before gc.collect() runs.
        del img, img_uint8

    except Exception as e:
        # -----------------------------------------------------------------
        # Error recovery for ANY unexpected failure
        # -----------------------------------------------------------------
        # ROBUSTNESS #1 — OOM recovery in except block
        # If a neuron triggers an OOM or any other error, we:
        #   1. Log it (don't crash)
        #   2. Force garbage collection + CUDA cache clear
        #   3. Continue to the next neuron
        neurons_failed += 1
        print(f"\n⚠️ Neuron {neuron_id} failed: {e}")
        wandb.log({"neuron": neuron_id, "error": str(e)})

        gc.collect()
        if device.type == "cuda":
            torch.cuda.empty_cache()
        continue

    # ---------------------------------------------------------------------
    # Periodic memory cleanup + GPU monitoring
    # ---------------------------------------------------------------------
    # ROBUSTNESS #1 + #7 — Memory cleanup and leak detection
    # Every `memory_cleanup_interval` neurons (default: 50), we:
    #   1. Run Python garbage collection (frees unreferenced objects)
    #   2. Release PyTorch's cached but unused GPU memory
    #   3. Log current GPU memory stats to W&B for leak detection
    #
    # If gpu/memory_reserved_gb keeps climbing while gpu/memory_allocated_gb
    # stays flat, you have a fragmentation issue.
    if neurons_completed % CONFIG["memory_cleanup_interval"] == 0:
        gc.collect()
        if device.type == "cuda":
            torch.cuda.empty_cache()
            wandb.log({
                "gpu/memory_allocated_gb": torch.cuda.memory_allocated() / 2**30,
                "gpu/memory_reserved_gb": torch.cuda.memory_reserved() / 2**30,
                "gpu/memory_peak_gb": torch.cuda.max_memory_allocated() / 2**30,
            })

    # ---------------------------------------------------------------------
    # Update progress bar postfix
    # ---------------------------------------------------------------------
    avg_time = time_optimization / neurons_completed if neurons_completed else 0
    pbar.set_postfix({
        "last": f"{neuron_time:.1f}s",
        "avg": f"{avg_time:.1f}s",
        "fail": neurons_failed,
    })

🧠 mixed5a:   0%|          | 0/832 [00:00<?, ?neuron/s]

100%|██████████| 1024/1024 [03:12<00:00,  5.31it/s]



⚠️ Neuron 449 failed: [Errno 2] No such file or directory: 'C:\\Users\\CATALU~1\\AppData\\Local\\Temp\\tmpf32cadjrwandb-media\\apj4d7g8.png'


100%|██████████| 1024/1024 [02:32<00:00,  6.70it/s]



⚠️ Neuron 450 failed: [Errno 2] No such file or directory: 'C:\\Users\\CATALU~1\\AppData\\Local\\Temp\\tmpf32cadjrwandb-media\\w1z555wa.png'


...


⚠️ Neuron 490 failed: [Errno 2] No such file or directory: 'C:\\Users\\CATALU~1\\AppData\\Local\\Temp\\tmpf32cadjrwandb-media\\lp6k954p.png'


100%|██████████| 1024/1024 [02:43<00:00,  6.26it/s]



⚠️ Neuron 491 failed: [Errno 2] No such file or directory: 'C:\\Users\\CATALU~1\\AppData\\Local\\Temp\\tmpf32cadjrwandb-media\\7vj7tliw.png'


 17%|█▋        | 178/1024 [00:35<02:51,  4.95it/s]


Interrupted optimization at step 179.

⚠️ Neuron 492 failed: [Errno 2] No such file or directory: 'C:\\Users\\CATALU~1\\AppData\\Local\\Temp\\tmpf32cadjrwandb-media\\vlyym9wp.png'


In [ ]:
# =============================================================================
# Cell 11: Run Summary & Cleanup
# =============================================================================
# Prints a human-readable summary of the completed run and logs
# final aggregate metrics to W&B before closing the run.
# =============================================================================

# --- Calculate totals ---
total_time = time.time() - t_start_total
throughput = neurons_completed / total_time if total_time > 0 else 0

# --- Print summary ---
print(f"\n{'=' * 60}")
print(f"  ✅ Run {run_id} Complete — {CONFIG['layer']}")
print(f"{'=' * 60}")
print(f"  Neurons completed:  {neurons_completed}")
print(f"  Neurons skipped:    {_skipped}")
print(f"  Neurons failed:     {neurons_failed}")
print(f"  Total time:         {total_time:.1f}s ({total_time / 3600:.1f}h)")
print(f"  Throughput:         {throughput:.2f} neurons/sec")
print()

# --- Timing breakdown ---
print(f"  ⏱️  Timing Breakdown:")
if total_time > 0:
    print(f"     Optimization:   {time_optimization:.1f}s "
          f"({100 * time_optimization / total_time:.1f}%)")
    print(f"     Saving:         {time_saving:.1f}s "
          f"({100 * time_saving / total_time:.1f}%)")

# --- GPU memory info ---
if device.type == "cuda":
    peak_mem = torch.cuda.max_memory_allocated() / 2**30
    print(f"  🖥️  Peak GPU memory: {peak_mem:.2f} GB")
print()
print(f"  📁 Results saved to: {save_dir}")

# --- Log final summary to W&B ---
wandb.log({
    "summary/neurons_completed": neurons_completed,
    "summary/neurons_skipped": _skipped,
    "summary/neurons_failed": neurons_failed,
    "summary/total_time_sec": total_time,
    "summary/throughput_neurons_per_sec": throughput,
    "summary/time_optimization_sec": time_optimization,
    "summary/time_saving_sec": time_saving,
})

# --- Close W&B run ---
wandb.finish()
print("✅ W&B run finished and synced")